In [1]:
#%matplotlib inline


import matplotlib as mpl
#%matplotlib notebook
mpl.use('qt5agg') 

import matplotlib.pyplot as plt
import cv2
import math
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import skimage as sk
from skimage.filters import roberts, sobel, scharr
from skimage import morphology
from skimage.measure import label

from skimage import data, util
from skimage.morphology import label

import scipy as sp
from scipy import *

import os
import shutil
import time

import glob

dt=0.1
dz=0.0038
scale=3.6481

#windowsize 1/res_valueにする。
res_value = 3

#mouse eventのときのresize
track_res=4

#ROIのwindowsizeをres_value_crop倍にする。
res_value_crop = 5

median_value = 9

#bobcat
#length_x=1608 #読み込み動画のサイズx
#length_y=1208 #読み込み動画のサイズy

#baumer
length_x=2048 #読み込み動画のサイズx
length_y=2048 #読み込み動画のサイズy
trash_size=20 #これ以下はごみとみなす
#Luca
#length_x=658 #読み込み動画のサイズx
#length_y=496 #読み込み動画のサイズy

#輝度値を0-255に比例配分する。
#int_min = 255
#int_max = 0

#for mac
#tgd = ["/Volumes/HD-PLFU3/federico/170827/170827_t13_d24p3_CP_29_700hpa0p5_leica_baumer_x20_10fps_025/"]

#for windows
tgd = ["G:/federico/170914/170914_t13_d24p3_CP_29_650hpa0p5_leica_baumer_x20_10fps_008/"]
#"F:/chiral_leica_exp/161009/t13_d25_s2/161009_t13_d25_s2_400hpa0p4s_leica_baumer_x20tox4_10fps_011/"]

In [2]:
#バッチ処理のループ開始。液滴選択→ﾄﾗｯｷﾝｸﾞ＋femto+フォルダ移動の順に行う。
for name in tgd :
    frames = np.loadtxt(name+"frames.txt", comments="pass", delimiter=",",usecols=(0,1))
        

    for p in range(1,len(frames)):#femto以外
        
        #座標取得
        frame_s = int(frames[p,0])
        frame_e = int(frames[p,1])

        REC=[]
        int_min = 255
        int_max = 0

        #ゴミ取りをした後の二値化動画を読み込み
        #cap = cv2.VideoCapture("label_cleaned.avi")
        start= frame_s
        img = cv2.imread(name+"tiff/%06d.tif" %start,0)
        img = cv2.resize(img,(length_x/track_res, length_y/track_res))
        cv2.namedWindow("mouse",cv2.WINDOW_AUTOSIZE)

        mouse_xy =[]
        def mouse_event(event,x,y,flags,param):

            if event == cv2.EVENT_LBUTTONUP:
                tmp = [int(x*track_res),int(y*track_res)]
                #print tmp
                #cv2.circle(img,(int(track_res*x),int(track_res*y)),20,(0,0,255),3)
                cv2.circle(img,(x,y),40,(0,0,255),3)
                mouse_xy.append(tmp)
                np.savetxt(name+"xy_track_%06dto%06d.txt" %(frame_s,frame_e), mouse_xy)
                #np.savetxt(name+"xy_track_used.txt", mouse_xy)

        cv2.setMouseCallback("mouse",mouse_event)

        while(True):
            cv2.imshow("mouse", img)
            if cv2.waitKey(1)&0xFF == ord("q"):
                cv2.imwrite(name+"droplets_%06dto%06d.tif" %(frame_s,frame_e), img)
                break
        cv2.destroyAllWindows()
        #cv2.DestroyWindow("mouse")

        #mouse_xy = np.loadtxt(name+"xy_track_%06dto%06d.txt" %(frame_s,frame_e))
        print mouse_xy
cv2.destroyAllWindows()

[[1024, 336]]
[[1056, 1648]]
[[1108, 1104]]


In [3]:
#バッチ処理のループ開始。ﾄﾗｯｷﾝｸﾞ＋femto+フォルダ移動の順に行う。

crop_size = 60 #(pix*2)^2で走査


#二値化の閾値は粒子が暗い時,コントラストがいい時は100、あかるくコントラストが悪いときは160くらいにするとよい
#Leicaで×20,int9のときは160にする。150913

thresh_bin_org = 120 #def =80, 161006
thresh_bin_new = 200 #二値化の閾値 x4でぼけてもいけるのは160くらい

#windowsize 1/res_valueにする。
res_value = 3

#mouse eventのときのresize
track_res=2

#ROIのwindowsizeをres_value_crop倍にする。
res_value_crop = 5

median_value = 9
for name in tgd:
    print name
    frames = np.loadtxt(name+"frames.txt", comments="pass", delimiter=",",usecols=(0,1))
    
    for p in range(1,len(frames)):#femto以外
        print p
        
        f = open(name+"conditions.txt","w")
        f.write("scale," +str(scale)+"\n")
        #f.write("THRESH_SIZE," +str(THRESH_SIZE)+"\n")
        f.write("thresh_bin_org," +str(thresh_bin_org)+"\n")
        f.write("median_value," +str(median_value)+"\n")
        f.write("crop_size," +str(crop_size))

        f.close()


        # In[80]:
        REC=glob.glob(name+"REC*")
        for i in range(0,len(REC)):
            if os.path.isfile(REC[i]) == True:
                os.remove(REC[i]) 


        if os.path.isfile(name+"z_target.txt") == True:
            os.remove(name+"z_target.txt")   
        if os.path.isfile(name+"movie.gif") == True:
            os.remove(name+"movie.gif")   

        if os.path.isfile(name+"z_target.txt") == True:
            os.remove(name+"z_target.txt") 

        if os.path.isfile(name+"*.jpeg") == True:
            os.remove(name+"*.jpeg")  

        if os.path.isfile(name+"REC.txt") == True:
            os.remove(name+"REC.txt") 

        if os.path.isfile(name+"REC_sort.txt") == True:
            os.remove(name+"REC_sort.txt") 

        if os.path.isfile(name+"tracking.png") == True:
            os.remove(name+"tracking.png") 

        if os.path.isdir(name+'img_med'):
            shutil.rmtree(name+"img_med") 


        if os.path.isdir(name+'img_bin'):
            shutil.rmtree(name+"img_bin") 


        if os.path.isdir(name+'img_fil'):
            shutil.rmtree(name+"img_fil") 


        if os.path.isdir(name+'img_src'):
            shutil.rmtree(name+"img_src") 


        if os.path.isdir(name+'crop'):
            shutil.rmtree(name+"crop") 

        if os.path.isdir(name+'img_src_ROI'):
            shutil.rmtree(name+"img_src_ROI") 

        if os.path.isdir(name+'condition.txt'):
            shutil.rmtree(name+"condition.txt") 

        if os.path.isdir(name+'tracking'):
            shutil.rmtree(name+"tracking") 

        if os.path.isdir(name+'tracking_all_ast'):
            shutil.rmtree(name+"tracking_all_ast") 

        if os.path.isdir(name+'tracking_all_seq'):
            shutil.rmtree(name+"tracking_all_seq")

        cv2.destroyAllWindows()


        if os.path.isdir(name+'img_med'):
            shutil.rmtree(name+"img_med") 

        os.mkdir(name+"img_med")

        if os.path.isdir(name+'img_bin'):
            shutil.rmtree(name+"img_bin") 

        os.mkdir(name+"img_bin")

        if os.path.isdir(name+'img_fil'):
            shutil.rmtree(name+"img_fil") 

        os.mkdir(name+"img_fil")

        if os.path.isdir(name+'img_src'):
            shutil.rmtree(name+"img_src") 

        os.mkdir(name+"img_src")

        if os.path.isdir(name+'img_src_ROI'):
            shutil.rmtree(name+"img_src_ROI") 

        os.mkdir(name+"img_src_ROI")

        if os.path.isdir(name+'img_src_ROI_norm'):
            shutil.rmtree(name+"img_src_ROI_norm") 

        os.mkdir(name+"img_src_ROI_norm")

        if os.path.isdir(name+'img_src_ROI_new'):
            shutil.rmtree(name+"img_src_ROI_new") 

        os.mkdir(name+"img_src_ROI_new")

        #座標取得
        frame_s = int(frames[p,0])
        frame_e = int(frames[p,1])
        
        mouse_xy = np.loadtxt(name+"xy_track_%06dto%06d.txt" %(frame_s,frame_e))
        mouse_xy = np.reshape(mouse_xy,(1,2))
        n_p = mouse_xy.shape


        start_time=time.time()
        cv2.destroyAllWindows()
        thresh_bin = thresh_bin_org
        REC=[]
        x_center_new=[]
        y_center_new=[]
        x_cm=[]
        y_cm=[]
        state=[]
        for j in range(n_p[0]): #全ての粒子に対して
            #REC=[]
            exec("REC_"+str(frame_s)+"to"+str(frame_e)+"_"+str(j)+"=[]")
            if os.path.isdir(name+"img_src_ROI/img_src_ROI_%06dto%06d_%02d" %(frame_s,frame_e,j)):
                shutil.rmtree(name+"img_src_ROI/img_src_ROI_%06dto%06d_%02d" %(frame_s,frame_e,j)) 

            os.mkdir(name+"img_src_ROI/img_src_ROI_%06dto%06d_%02d" %(frame_s,frame_e,j))

            if os.path.isdir(name+"img_src_ROI_new/img_src_ROI_new_%06dto%06d_%02d" %(frame_s,frame_e,j)):
                shutil.rmtree(name+"img_src_ROI_new/img_src_ROI_new_%06dto%06d_%02d" %(frame_s,frame_e,j)) 

            os.mkdir(name+"img_src_ROI_new/img_src_ROI_new_%06dto%06d_%02d" %(frame_s,frame_e,j))
            if os.path.isdir(name+"img_src_ROI_norm/img_src_ROI_norm_%06dto%06d_%02d" %(frame_s,frame_e,j)):
                shutil.rmtree(name+"img_src_ROI_norm/img_src_ROI_norm_%06dto%06d_%02d" %(frame_s,frame_e,j)) 

            os.mkdir(name+"img_src_ROI_norm/img_src_ROI_norm_%06dto%06d_%02d" %(frame_s,frame_e,j))

            if os.path.isdir(name+"img_med/img_med_%06dto%06d_%02d" %(frame_s,frame_e,j)):
                shutil.rmtree(name+"img_med/img_med_%06dto%06d_%02d" %(frame_s,frame_e,j)) 

            os.mkdir(name+"img_med/img_med_%06dto%06d_%02d" %(frame_s,frame_e,j))

            if os.path.isdir(name+"img_bin/img_bin_%06dto%06d_%02d" %(frame_s,frame_e,j)):
                shutil.rmtree(name+"img_bin/img_bin_%06dto%06d_%02d" %(frame_s,frame_e,j)) 

            os.mkdir(name+"img_bin/img_bin_%06dto%06d_%02d" %(frame_s,frame_e,j))

            if os.path.isdir(name+"img_fil/img_fil_%06dto%06d_%02d" %(frame_s,frame_e,j)):
                shutil.rmtree(name+"img_fil/img_fil_%06dto%06d_%02d" %(frame_s,frame_e,j)) 

            os.mkdir(name+"img_fil/img_fil_%06dto%06d_%02d" %(frame_s,frame_e,j))

            state=np.append(state,"success")

        #print state
        #for root, dir, files in os.walk("tiff"):#Pos0フォルダのファイルを探す。
        for i in range(frame_s,frame_e+1):#frame_sからframe_eまで
            #print i
            img_src = cv2.imread(name+"tiff/%06d.tif" %i,0)
            #indexは粒子番号となるようなROI用の配列を用意する
            x_min=[]
            x_max=[]
            y_min=[]
            y_max=[]
            for j in range(n_p[0]): #全ての粒子に対して
                if state[j] == "failure":
                    x_min=np.append(x_min,x_center_new[j] - crop_size)
                    x_max=np.append(x_max,x_center_new[j] + crop_size)
                    y_min=np.append(y_min,y_center_new[j] - crop_size)
                    y_max=np.append(y_max,y_center_new[j] + crop_size)
                else:
                    #x_minは縦
                    if i==frame_s:
                        x_center_new=np.append(x_center_new,mouse_xy[j,1])
                        y_center_new=np.append(y_center_new,mouse_xy[j,0])
                        x_cm=np.append(x_cm,x_center_new[j])
                        y_cm=np.append(y_cm,y_center_new[j])

                    x_min=np.append(x_min,x_center_new[j] - crop_size)
                    x_max=np.append(x_max,x_center_new[j] + crop_size)
                    y_min=np.append(y_min,y_center_new[j] - crop_size)
                    y_max=np.append(y_max,y_center_new[j] + crop_size)

                    if x_min[j]<0+1 or x_max[j]>length_y-1 or y_min[j]<0+1 or y_max[j]>length_x-1:
                        state[j]= "failure"
                    else:
                    
                        x_min = np.array(x_min, dtype=int)
                        x_max = np.array(x_max, dtype=int)
                        y_min = np.array(y_min, dtype=int)
                        y_max = np.array(y_max, dtype=int)
                        
                        ROI = img_src[x_min[j]:x_max[j], y_min[j]:y_max[j]]
                        #cv2.namedWindow("ROI"+str(j),cv2.WINDOW_AUTOSIZE)
                        cv2.imwrite(name+"img_src_ROI/img_src_ROI_%06dto%06d_%02d/img_src_ROI_%06d.tif" %(frame_s,frame_e,j,i), ROI)
                        #ROI_res = cv2.resize(ROI,(crop_size*2*res_value_crop, crop_size*2*res_value_crop))

                        #cv2.imshow("ROI"+str(j), ROI_res)
                        #key = cv2.waitKey(10)

                        #画像のコントラストを良くする。
                        ROI = cv2.imread(name+"img_src_ROI/img_src_ROI_%06dto%06d_%02d/img_src_ROI_%06d.tif" %(frame_s,frame_e,j,i),0)

                        int_min_tmp = np.min(ROI)
                        int_max_tmp = np.max(ROI)
                        ROI_norm = (ROI-int_min_tmp)*255.0/(int_max_tmp-int_min_tmp) #

                        ROI_int = np.uint8(ROI_norm)
                        cv2.imwrite(name+"img_src_ROI_norm/img_src_ROI_norm_%06dto%06d_%02d/img_src_ROI_norm_%06d.tif" %(frame_s,frame_e,j,i), ROI_int)


                        #ROI_norm_res = cv2.resize(ROI_int,(crop_size*2*res_value_crop, crop_size*2*res_value_crop))
                        #cv2.imshow('ROI_norm'+str(j), ROI_norm_res)
                        #key = cv2.waitKey(10)


                        #9*9のmedian filter
                        #medianなくす
                        img_med = sp.ndimage.filters.median_filter(ROI_int,median_value) #9*9のmedian filter
                        img_med = cv2.convertScaleAbs(img_med) 
                        cv2.imwrite(name+"img_med/img_med_%06dto%06d_%02d/img_med_%06d.tif" %(frame_s,frame_e,j,i), img_med)

                        #cv2.namedWindow("img_med"+str(j),cv2.WINDOW_AUTOSIZE)
                        #img_med_res = cv2.resize(img_med,(crop_size*2*res_value_crop, crop_size*2*res_value_crop))
                        #cv2.imshow('img_med'+str(j), img_med_res)
                        #key = cv2.waitKey(10)




                        #median filterをかけた動画を読み込み、binarizeする。
                        retreval, img_bin = cv2.threshold(img_med, thresh_bin,255,cv2.THRESH_BINARY_INV)
                        img_bin = cv2.convertScaleAbs(img_bin)
                        cv2.imwrite(name+"img_bin/img_bin_%06dto%06d_%02d/img_bin_%06d.tif" %(frame_s,frame_e,j,i), img_bin)

                        #cv2.namedWindow("img_bin"+str(j),cv2.WINDOW_AUTOSIZE)
                        #img_bin_res = cv2.resize(img_bin,(crop_size*2*res_value_crop, crop_size*2*res_value_crop))
                        #cv2.imshow('img_bin'+str(j), img_bin_res)
                        #key = cv2.waitKey(10)
                        #cv2.imwrite("./img_bin/img_bin_%04d.tif" %i, img_bin)
                        #rec1.write(img_bin)

                        #binary fill holeをかける。0,1の画像で出るから255をかけて8bit表示で白黒になるように設定している。
                        img_fil=255*sp.ndimage.binary_fill_holes(img_bin).astype(ROI.dtype)

                        cv2.imwrite(name+"/img_fil/img_fil_%06dto%06d_%02d/img_fil_%06d.tif" %(frame_s,frame_e,j,i), img_fil)

                        cv2.namedWindow("fil"+str(j),cv2.WINDOW_AUTOSIZE)
                        img_fil_res = cv2.resize(img_fil,(crop_size*2*res_value_crop, crop_size*2*res_value_crop))
                        cv2.imshow('fil'+str(j), img_fil_res)

                        #ret, temp = cap.read()
                        temp = cv2.imread(name+"img_fil/img_fil_%06dto%06d_%02d/img_fil_%06d.tif" %(frame_s,frame_e,j,i),0)
                        label_im, n = sp.ndimage.label(temp) #label_imは粒子があるところにラベル番号が入れられているもの、nは総ラベル数
                        #重心を求めてcmにlist of toupleで出力
                        cm = sp.ndimage.measurements.center_of_mass(temp, label_im, range(1,n+1)) #cmはlist of touple
                        #print cm
                        label_im_temp = np.copy(label_im) #nd.arrayでは同じものを参照しているので、代入はきをつけないといけない。
                        #regionpropsで粒子の特性をpropsに出力。
                        props= sk.measure.regionprops(label_im_temp)

                        target = 0
                        if n==0:
                            thresh_bin=thresh_bin_new
                        elif n==1:
                            #小さい過ぎたらゴミだと判断
                            if props[0].area < trash_size:
                                state[j]="failure"
                            else:
                                thresh_bin = thresh_bin_org
                                #x_cmは実数の座標、x_center_newは表示用の整数値
                                x_cm[j] = x_min[j]+cm[0][0]
                                y_cm[j] = y_min[j]+cm[0][1]
                                x_center_new[j] = x_min[j]+round(cm[0][0])
                                y_center_new[j] = y_min[j]+round(cm[0][1])
                        else:
                            thresh_bin = thresh_bin_org
                            target=-1

                            for l in range(n):
                                if props[l].area > trash_size:
                                    target = l
                            dis_sq =((crop_size-cm[target][0])**2+(crop_size-cm[target][1])**2)
                            if target ==-1:#ごみしかなかったら終わり
                                state[j]="failure"
                            for l in range(n):
                                if ((crop_size-cm[l][0])**2+(crop_size-cm[l][1])**2)<dis_sq:
                                    target = l #粒子番号の定義を１からにしたので
                            #x_cmは実数の座標、x_center_newは表示用の整数値
                            x_cm[j] = x_min[j]+cm[target][0]
                            y_cm[j] = y_min[j]+cm[target][1]
                            x_center_new[j] = x_min[j]+round(cm[target][0])
                            y_center_new[j] = y_min[j]+round(cm[target][1])
                        #実際の重心でROIをきった画像
                        if x_center_new[j]<0+crop_size+1 or x_center_new[j]>length_y-crop_size-1 or y_center_new[j]<0+crop_size+1 or y_center_new[j]>length_x-crop_size-1:
                            state[j]= "failure"
                        elif n!=0:
                    
                            x_center_new=np.array(x_center_new, dtype=int)
                            y_center_new=np.array(y_center_new, dtype=int)
                            ROI_new = img_src[(x_center_new[j]-crop_size):(x_center_new[j]+crop_size), (y_center_new[j]-crop_size):(y_center_new[j]+crop_size)]
                            cv2.namedWindow("new"+str(j),cv2.WINDOW_AUTOSIZE)
                            cv2.imwrite(name+"img_src_ROI_new/img_src_ROI_new_%06dto%06d_%02d/img_src_ROI_new_%06d.tif" %(frame_s,frame_e,j,i), ROI_new)
                            ROI_new_res = cv2.resize(ROI_new,(crop_size*2*res_value_crop, crop_size*2*res_value_crop))

                            cv2.imshow("new"+str(j), ROI_new_res)
                            key = cv2.waitKey(10)
                        exec("REC_"+str(frame_s)+"to"+str(frame_e)+"_"+str(j)+".append([j,i,x_cm[j],y_cm[j]])")
                        #filename.append([j,i,x_cm[j], y_cm[j]])
                        #解析開始時間の50s後に20s測定して2um/s以下ならゴミとみなす 200frameで20pix
                        #if i > frame_s+500 and ((REC[i-frame_s][2]-REC[i-frame_s-200][2])**2+(REC[i-frame_s][3]-REC[i-frame_s-200][3])**2)<400:
                            #print str(j)+"=gomi"
                            #break
        for j in range(n_p[0]):
            
            exec("np.savetxt(name+\"REC_%06dto%06d_%02d.txt\" %(frame_s,frame_e,j), REC_"+str(frame_s)+"to"+str(frame_e)+"_"+str(j)+")")

        cv2.destroyAllWindows()
        elapsed_time = time.time() - start_time
        #print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        #処理にかかった時間を出力
        f=open(name+"time.txt","w")
        time_fs = str(elapsed_time)
        f.write(time_fs)
        f.close()

        #ここからやる10/11
        if os.path.isdir(name+'tracking_all_ast_test'):
            shutil.rmtree(name+"tracking_all_ast_test") 

        os.mkdir(name+"tracking_all_ast_test")

        #for root, dir, files in os.walk("img_src"):#Pos0フォルダのファイルを探す。
        for l in range(frame_s,frame_s+1):
            img_src = cv2.imread(name+"tiff/%06d.tif" %l,0)

            temp = cv2.cvtColor(img_src, cv2.COLOR_GRAY2BGR)

            for i in range(0,n_p[0]): #粒子iにたいして
                #print i
                xy = []
                cm_p = np.loadtxt(name+"REC_%06dto%06d_%02d.txt" %(frame_s,frame_e,i))
                shape = cm_p.shape
                #print shape
                #print int(cm_p[-1][1])+1-int(cm_p[0][1])
                if shape[0]!=0:
                    xy = cm_p[:,[2,3]]+0.5
                    xy_round = np.array(xy, dtype = np.uint32)
                    for j in range(0, int(cm_p[-1][1])+1-int(cm_p[0][1])):
                        if j == 0:
                            if i%20 == 0:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(0,0,200),2)
                            if i%20 == 1:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(200,0,0),2)
                            if i%20 == 2:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(0,600,600),2)
                            if i%20 == 3:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(200,200,0),2)
                            if i%20 == 4:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(200,500,500),2)
                            if i%20 == 5:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(200,0,200),2)
                            if i%20 == 6:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(100,0,50),2)
                            if i%20 == 7:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(300,0,0),2)
                            if i%20 == 8:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(0,500,100),2)
                            if i%20 == 9:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(500,100,100),2)
                            if i%20 == 10:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(200,0,500),2)
                            if i%20 == 11:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(200,0,0),2)
                            if i%20 == 12:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(500,500,500),2)
                            if i%20 == 13:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(300,200,0),2)
                            if i%20 == 14:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(500,500,200),2)
                            if i%20 == 15:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(200,0,200),2)
                            if i%20 == 16:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(0,300,500),2)
                            if i%20 == 17:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(300,0,0),2)
                            if i%20 == 18:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(300,0,300),2)
                            if i%20 == 19:
                                cv2.putText(temp,str(i),(int(xy_round[j][1]),int(xy_round[j][0])),cv2.FONT_HERSHEY_TRIPLEX,2,(0,0,0),2)

                        else :
                        #print j
                        #色BGR
                            if i%20 == 0:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(0,0,200),2)
                            if i%20 == 1:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(200,0,0),2)
                            if i%20 == 2:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(0,600,600),2)
                            if i%20 == 3:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(200,200,0),2)
                            if i%20 == 4:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(200,500,500),2)
                            if i%20 == 5:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(200,0,200),2)
                            if i%20 == 6:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(100,0,50),2)
                            if i%20 == 7:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(300,0,0),2)
                            if i%20 == 8:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(0,500,100),2)
                            if i%20 == 9:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(500,100,100),2)
                            if i%20 == 10:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(200,0,500),2)
                            if i%20 == 11:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(200,0,0),2)
                            if i%20 == 12:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(500,500,500),2)
                            if i%20 == 13:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(300,200,0),2)
                            if i%20 == 14:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(500,500,200),2)
                            if i%20 == 15:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(200,0,200),2)
                            if i%20 == 16:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(0,300,500),2)
                            if i%20 == 17:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(300,0,0),2)
                            if i%20 == 18:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(300,0,300),2)
                            if i%20 == 19:
                                cv2.line(temp,(int(xy_round[j][1]),int(xy_round[j][0])),(int(xy_round[j-1][1]),int(xy_round[j-1][0])),(0,0,0),2)
            cv2.imwrite(name+"tracking_all_ast_test/tracking_all_ast_test_%06dto%06d.tif" %(frame_s,frame_e), temp)
            cv2.namedWindow("tracking_all",cv2.WINDOW_AUTOSIZE)
            temp_res = cv2.resize(temp,(length_x/2, length_y/2))        

            cv2.imshow('tracking_all', temp_res)
            key = cv2.waitKey(10)
            #cv2.imwrite("./tracking_all_ast/tracking_all_ast_%04d.tif" %k, temp)
            #rec5.write(temp)


        cv2.destroyAllWindows()


        dir_name = str(frame_s)+'to'+str(frame_e)
        if os.path.isdir(name+dir_name):
            shutil.rmtree(name+dir_name) 

        os.mkdir(name+dir_name)
        # In[80]:
        REC=glob.glob(name+"REC*")
        for i in range(0,len(REC)):
            shutil.move(REC[i], name+dir_name)

        #shutil.move("crop", dir_name)
        #shutil.move("label_clean", dir_name)
        #shutil.move("tracking", dir_name)
        shutil.move(name+"tracking_all_ast_test", name+dir_name)
        #shutil.move(name+"tracking_all_ast_chosen", name+dir_name)
        #shutil.move("tracking_all_ast", dir_name)
        #shutil.move("tracking_all_seq", dir_name)
        shutil.move(name+"img_med", name+dir_name)
        shutil.move(name+"img_bin", name+dir_name)
        shutil.move(name+"img_fil", name+dir_name)
        shutil.move(name+"img_src", name+dir_name)
        shutil.move(name+"img_src_ROI", name+dir_name)
        shutil.move(name+"img_src_ROI_norm", name+dir_name)
        shutil.move(name+"img_src_ROI_new", name+dir_name)
        shutil.move(name+"droplets_%06dto%06d.tif" %(frame_s,frame_e), name+dir_name)
        #shutil.move("z_target.txt", dir_name)
        #shutil.move("tracking.png", dir_name)
        #shutil.move("intensity.png", dir_name)
        shutil.move(name+"xy_track_%06dto%06d.txt" %(frame_s,frame_e), name+dir_name)
        shutil.move(name+"conditions.txt", name+dir_name)
        shutil.move(name+"time.txt", name+dir_name)
        
    
    

    #femtojet動画をつくる
    frame_s2 = int(frames[0,0])
    frame_e2 = int(frames[0,1])
    if os.path.isdir(name+"femto") == False:
        os.mkdir(name+"femto")
        for i in range(frame_s2,frame_e2+1):
            femto = cv2.imread(name+"tiff/%06d.tif" %i)
            cv2.imwrite(name+"femto/femto_%06d.tif" %i, femto)
            cv2.namedWindow("femto",cv2.WINDOW_AUTOSIZE)
            femto_res = cv2.resize(femto,(length_x/res_value, length_y/res_value))
            cv2.imshow('femto_res', femto_res)
            key = cv2.waitKey(10)
    cv2.destroyAllWindows()   

G:/federico/170914/170914_t13_d24p3_CP_29_650hpa0p5_leica_baumer_x20_10fps_008/
1
2
3


In [4]:
#xyzのファイルからうまくログが取れているフレームを抜き取り、整形しファイルに吐き出す。
get_ipython().magic(u'matplotlib')

plt.rcParams["font.size"]=20
from mpl_toolkits.mplot3d import Axes3D


import matplotlib.cm as cm

for name in tgd: 
    
    frames = np.loadtxt(name+"frames.txt", comments="pass", delimiter=",",usecols=(0,1))
    filename=glob.glob(name+"*_xyz.txt")#xyzのデータファイルをのパスをfilenameに取得
    fh = open(filename[0])
    print filename[0]
    line = fh.readlines()

    print len(line)
    #改行コードを取り除く
    for i in range(len(line)):
        if ".tif" in line[i]:
            #print line[i]
            #print line[i]
            #line[i]=line[i].strip(".tif\r\n")# for mac
            line[i]=line[i].strip(".tif\n")# for windows
            line[i]=line[i].strip(".tif")
            line[i]=line[i].strip(".tif")
            #print line[i]
        elif "71023" in line[i]:
            #print line[i]
            line[i]=line[i].strip("71023")
            #line[i]=line[i].strip("\r\n") #for mac
            line[i]=line[i].strip("\n") #for win
            line[i]=line[i].strip(" ")
        #elif line[i]=="\r\n":#for mac
        elif line[i]=="\n": #for win
            
            pass
        else:
            #print line[i]
            #line[i]=line[i].strip("\r\n")#for mac
            line[i]=line[i].strip("\n")#for win
    #うまくログが取れているframeだけをとりのぞく
    #line_newにframe number x y z
    count =0
    line_norm=[]
    for i in range(len(line)):
        count += 1
        #print count
        #print line[i]
        #print count
        
        #if line[i]=="\r\n": #editted on 170828 for mac
        if line[i]=="\n": #editted on 170828 for win
            #print line[i]
            if count ==5 and line[i-1]!="99999":
                #print line[i]
                #line_norm=np.append(line_norm,[line[i-4],line[i-4],line[i-3],line[i-2],line[i-1]])
                line_norm=np.append(line_norm,[line[i-4],line[i-4],line[i-3],line[i-2],line[i-1]]) # 170828 do not mind line[-4] is doubled.
                #print line[i-1]
            count=0
    #print line_norm
    line_norm=np.array(line_norm,dtype=float)

    line_new=np.reshape(line_norm,(int(len(line_norm)/5),5))
    line_new[:,1]=line_new[:,1]*dt
    line_new[:,4]=line_new[:,4]*dz
    
    np.savetxt(name+"xyz_normal.txt",line_new)
    
    xyz_sum=[]
    for p in range(1,len(frames)):#femto以外
        #print p
        #座標取得
        frame_s = int(frames[p,0])
        frame_e = int(frames[p,1])
        
        dir_name = str(frame_s)+'to'+str(frame_e)
        xyz_norm=[]
        xy_rel=np.loadtxt(name+dir_name+"/REC_%06dto%06d_00.txt" %(frame_s,frame_e))
        #print xy_rel.dtype
        for q in range(frame_s,frame_e+1):
            tmp= np.where(line_new[:,0]==float(q))
            #print len(tmp[0])
            if len(tmp[0]) !=0 and len(xy_rel) !=0:
                
                if int(xy_rel[-1,1])>=q:
                #print q
                #print tmp[0]
                    xyz_norm=np.append(xyz_norm,[q,q*dt,-line_new[tmp[0],2]+xy_rel[q-frame_s,2]/scale,-line_new[tmp[0],3]+xy_rel[q-frame_s,3]/scale,line_new[tmp[0],4]])
                    xyz_sum=np.append(xyz_sum,[q,q*dt,-line_new[tmp[0],2]+xy_rel[q-frame_s,2]/scale,-line_new[tmp[0],3]+xy_rel[q-frame_s,3]/scale,line_new[tmp[0],4]])
        xyz_norm=np.reshape(xyz_norm,(int(len(xyz_norm)/5),5))
        np.savetxt(name+dir_name+"/xyz_partial.txt",xyz_norm)
        
    xyz_sum=np.reshape(xyz_sum,(int(len(xyz_sum)/5),5))
    np.savetxt(name+"xyz_summary.txt",xyz_sum)  
    

    #zインデックスはframe_extの最小値から最大値まで


    
    
    fig1 = plt.figure(figsize=(20,20))
    ax1 = Axes3D(fig1)

    fig2 = plt.figure(figsize=(20,20))
    ax2 = Axes3D(fig2)
    colmap= (xyz_sum[:,0]-xyz_sum[0,0])*255/(xyz_sum[-1,0]-xyz_sum[0,0])

    ax1.set_xlabel("X[um]")
    ax1.set_ylabel("Y[um]")
    ax1.set_zlabel("Z[um]")
    ax1.set_aspect("equal","box")

    ax1.set_title("Reconstructed trajectory")

    ax2.set_xlabel("X[um]")
    ax2.set_ylabel("Y[um]")
    ax2.set_zlabel("Z[um]")
    ax2.set_aspect("equal","box")

    ax2.set_title("Reconstructed trajectory")


    #dropletのxyz座標 targetのxy座標を取り出す。
    X = xyz_sum[:,2]
    Y = xyz_sum[:,3]
    Z = xyz_sum[:,4]

    #asplectratioを合わせるためのテク　透明な立方体の箱をつくる
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())

    # Comment or uncomment following both lines to test the fake bounding box:
    for xb, yb, zb in zip(Xb, Yb, Zb):   
        ax1.plot([xb], [yb], [zb], 'w')
        ax2.plot([xb], [yb], [zb], 'w')

    ax2.scatter(X, Y, Z, facecolor ="none", edgecolor="blue", s= 100, marker="o")
    ax1.scatter(X, Y, Z,c=colmap,cmap="jet", s= 300, marker="o",edgecolor="black",alpha=0.2)
    fig1.savefig(name+"trajectory_color.png")
    fig2.savefig(name+"trajectory_blue.png")
    #plt.show()
    fig1.show()
    fig2.show()
    #plt.close(fig1)
    #plt.close(fig2)
    #2D trajecotry
    fig0 = plt.figure(figsize=(20,20))
    ax0=fig0.add_subplot(1,1,1)
    ax0.set_xlabel("Y(um)",size=30)
    ax0.set_ylabel("X(um)",size=30)
    ax0.scatter(Y,-X,c=colmap,s=100,marker=".",edgecolor="none",label="trajectory")
    ax0.set_title("trajectory")
    ax0.legend
    ax0.set_aspect("equal")
    fig0.savefig(name+"2D_trajectory.png")
    #fig0.show()
    
    

Using matplotlib backend: Qt5Agg
G:/federico/170914/170914_t13_d24p3_CP_29_650hpa0p5_leica_baumer_x20_10fps_008\170914_t13_d24p3_CP_29_650hpa0p5_leica_baumer_x20_10fps_008_xyz.txt
27014
